In [40]:
import pandas as pd
import requests
import json
import openai
from langchain_openai import ChatOpenAI
import os
from CredentialTo.CredentialToBrokerAPI import CredentialUpstox
from SmartApi.smartConnect import SmartConnect
import settings
from datetime import datetime
import upstox_client 



In [41]:
df_futureOptions = pd.DataFrame()
masterData = pd.read_json("https://assets.upstox.com/market-quote/instruments/exchange/NSE.json.gz")
# masterData = requests.get(url).json() 
df_futureOptions = pd.DataFrame.from_dict(masterData)

In [42]:
df_futureOptions.head()

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,last_trading_date,price_quote_unit
0,0.0,NSE_FO,BANKNIFTY,NSE,1.764095e+12,PE,BANKNIFTY,BANKNIFTY,NSE_FO|49906,35.0,...,INDEX,INDEX,BANKNIFTY 54900 PE 25 NOV 25,54900.0,1.0,NaN,NaN,NaN,NaN,NaN
1,0.0,NSE_FO,BANKNIFTY,NSE,1.764095e+12,CE,BANKNIFTY,BANKNIFTY,NSE_FO|49907,35.0,...,INDEX,INDEX,BANKNIFTY 55000 CE 25 NOV 25,55000.0,1.0,NaN,NaN,NaN,NaN,NaN
2,0.0,NSE_FO,BANKNIFTY,NSE,1.764095e+12,PE,BANKNIFTY,BANKNIFTY,NSE_FO|49904,35.0,...,INDEX,INDEX,BANKNIFTY 54800 PE 25 NOV 25,54800.0,1.0,NaN,NaN,NaN,NaN,NaN
3,0.0,NSE_FO,BANKNIFTY,NSE,1.764095e+12,CE,BANKNIFTY,BANKNIFTY,NSE_FO|49905,35.0,...,INDEX,INDEX,BANKNIFTY 54900 CE 25 NOV 25,54900.0,1.0,NaN,NaN,NaN,NaN,NaN
4,0.0,NCD_FO,JPYINR,NSE,1.774636e+12,CE,JPYINR,JPYINR,NCD_FO|14294,1.0,...,CUR,CUR,JPYINR 61 CE 27 MAR 26,61.0,1000.0,NaN,NaN,NaN,NaN,NaN


Convert Expiry to YYYY-MM-DD format

In [43]:
# Convert expiry from epoch (ms) to datetime
df_futureOptions["expiry"] = pd.to_datetime(df_futureOptions["expiry"], unit="ms")

# Convert datetime to string format YYYY-MM-DD
df_futureOptions["expiry"] = df_futureOptions["expiry"].dt.strftime("%Y-%m-%d")


# print(df_futureOptions.dtypes)  # verify expiry is now string
df_futureOptions.head()

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,last_trading_date,price_quote_unit
0,0.0,NSE_FO,BANKNIFTY,NSE,2025-11-25,PE,BANKNIFTY,BANKNIFTY,NSE_FO|49906,35.0,...,INDEX,INDEX,BANKNIFTY 54900 PE 25 NOV 25,54900.0,1.0,NaN,NaN,NaN,NaN,NaN
1,0.0,NSE_FO,BANKNIFTY,NSE,2025-11-25,CE,BANKNIFTY,BANKNIFTY,NSE_FO|49907,35.0,...,INDEX,INDEX,BANKNIFTY 55000 CE 25 NOV 25,55000.0,1.0,NaN,NaN,NaN,NaN,NaN
2,0.0,NSE_FO,BANKNIFTY,NSE,2025-11-25,PE,BANKNIFTY,BANKNIFTY,NSE_FO|49904,35.0,...,INDEX,INDEX,BANKNIFTY 54800 PE 25 NOV 25,54800.0,1.0,NaN,NaN,NaN,NaN,NaN
3,0.0,NSE_FO,BANKNIFTY,NSE,2025-11-25,CE,BANKNIFTY,BANKNIFTY,NSE_FO|49905,35.0,...,INDEX,INDEX,BANKNIFTY 54900 CE 25 NOV 25,54900.0,1.0,NaN,NaN,NaN,NaN,NaN
4,0.0,NCD_FO,JPYINR,NSE,2026-03-27,CE,JPYINR,JPYINR,NCD_FO|14294,1.0,...,CUR,CUR,JPYINR 61 CE 27 MAR 26,61.0,1000.0,NaN,NaN,NaN,NaN,NaN


Fetch NEarest Expiry date from Today

In [44]:


# Today as Timestamp
today = pd.to_datetime(datetime.today().date())

# Convert expiry_dates to datetime
expiry_dates = pd.to_datetime(df_futureOptions["expiry"], errors="coerce")

# Filter only future expiries
future_expiries = expiry_dates[expiry_dates >= today]

# Find nearest expiry
nearest_expiry = future_expiries.min()

print("Nearest expiry:", nearest_expiry)


Nearest expiry: 2025-09-09 00:00:00


In [45]:
settings.expiry_list_upstox[0]

'2025-09-09'

Filter df_futureOptions BY : ["name"] == "NIFTY" | ["segment"] == "NSE_FO" | expiry

In [46]:
df_futureOptions = df_futureOptions[ (df_futureOptions["name"] == "NIFTY") & (df_futureOptions["segment"] == "NSE_FO")  & (df_futureOptions["expiry"] == settings.expiry_list_upstox[0]) ]
df_futureOptions =  df_futureOptions.reset_index(drop=True)
df_futureOptions

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,last_trading_date,price_quote_unit
0,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40700,75.0,...,INDEX,INDEX,NIFTY 26850 PE 09 SEP 25,26850.0,1.0,NaN,NaN,NaN,NaN,NaN
1,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40650,75.0,...,INDEX,INDEX,NIFTY 25750 PE 09 SEP 25,25750.0,1.0,NaN,NaN,NaN,NaN,NaN
2,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40652,75.0,...,INDEX,INDEX,NIFTY 25800 PE 09 SEP 25,25800.0,1.0,NaN,NaN,NaN,NaN,NaN
3,1.0,NSE_FO,NIFTY,NSE,2025-09-09,CE,NIFTY,NIFTY,NSE_FO|40651,75.0,...,INDEX,INDEX,NIFTY 25800 CE 09 SEP 25,25800.0,1.0,NaN,NaN,NaN,NaN,NaN
4,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40654,75.0,...,INDEX,INDEX,NIFTY 25850 PE 09 SEP 25,25850.0,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40478,75.0,...,INDEX,INDEX,NIFTY 23050 PE 09 SEP 25,23050.0,1.0,NaN,NaN,NaN,NaN,NaN
168,1.0,NSE_FO,NIFTY,NSE,2025-09-09,CE,NIFTY,NIFTY,NSE_FO|40480,75.0,...,INDEX,INDEX,NIFTY 23100 CE 09 SEP 25,23100.0,1.0,NaN,NaN,NaN,NaN,NaN
169,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40483,75.0,...,INDEX,INDEX,NIFTY 23100 PE 09 SEP 25,23100.0,1.0,NaN,NaN,NaN,NaN,NaN
170,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40465,75.0,...,INDEX,INDEX,NIFTY 23000 PE 09 SEP 25,23000.0,1.0,NaN,NaN,NaN,NaN,NaN


Fetch NIFTY 50 instrument DATA

In [47]:
# Filter NIFTY 50 data
df_nifty50 = df_futureOptions[
    (df_futureOptions["instrument_key"] == "NSE_INDEX|Nifty 50")]

df_nifty50

,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,last_trading_date,price_quote_unit


Fetch NIFTY LTP 

In [48]:
instrument_key='NSE_INDEX|Nifty 50'

df_ltp = pd.DataFrame



from upstox_client.rest import ApiException
from pprint import pprint

# Configure OAuth2 access token for authorization: OAUTH2
configuration = upstox_client.Configuration()
configuration.access_token = CredentialUpstox.ACCESS_TOKEN  
# create an instance of the API class
api_instance = upstox_client.MarketQuoteV3Api(upstox_client.ApiClient(configuration))

try:
    # Market quotes and instruments - LTP quotes.
    api_response = api_instance.get_ltp(instrument_key=instrument_key)
    
    #  Convert API response to dict
    data_dict = api_response.to_dict()
    
      # Convert to DataFrame
    df_ltp = df_ltp.from_dict(data_dict["data"], orient="index").reset_index()
    df_ltp.rename(columns={"index": "instrument_key"}, inplace=True)
    print(df_ltp)
    
    # pprint(api_response)
except ApiException as e:
    print("Exception when calling MarketQuoteV3Api->get_ltp: %s\n" % e)

       instrument_key  last_price    instrument_token  ltq  volume       cp
0  NSE_INDEX:Nifty 50     24741.0  NSE_INDEX|Nifty 50    0       0  24734.3


Filter By Strike Price

In [49]:

# Extract LTP from response dict
nifty_ltp_value = float(df_ltp["last_price"])

import math

# nifty50_ltp_data = 24740.6

# Round to nearest 50
nearest_strike = round(nifty_ltp_value / 50) * 50  

# Generate 5 strikes around LTP (2 below, 2 above, + the nearest one)
strike_list = [nearest_strike + i * 50 for i in range(-3, 3)]

# Filter DataFrame
df_futureOptions = df_futureOptions[
    df_futureOptions["strike_price"].isin(strike_list)
].reset_index(drop=True)

print("LTP:", nifty_ltp_value)
print("Nearest 5 strikes:", strike_list)


LTP: 24741.0
Nearest 5 strikes: [24600, 24650, 24700, 24750, 24800, 24850]


/var/folders/bm/2sqnclys19zb2dh55g0cx4zc0000gn/T/ipykernel_88512/701431457.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  nifty_ltp_value = float(df_ltp["last_price"])


FINAL MASTERDATA

In [50]:
df_futureOptions.head(11)


,weekly,segment,name,exchange,expiry,instrument_type,asset_symbol,underlying_symbol,instrument_key,lot_size,...,asset_type,underlying_type,trading_symbol,strike_price,qty_multiplier,isin,security_type,short_name,last_trading_date,price_quote_unit
0,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40573,75.0,...,INDEX,INDEX,NIFTY 24650 PE 09 SEP 25,24650.0,1.0,NaN,NaN,NaN,NaN,NaN
1,1.0,NSE_FO,NIFTY,NSE,2025-09-09,CE,NIFTY,NIFTY,NSE_FO|40572,75.0,...,INDEX,INDEX,NIFTY 24650 CE 09 SEP 25,24650.0,1.0,NaN,NaN,NaN,NaN,NaN
2,1.0,NSE_FO,NIFTY,NSE,2025-09-09,CE,NIFTY,NIFTY,NSE_FO|40577,75.0,...,INDEX,INDEX,NIFTY 24700 CE 09 SEP 25,24700.0,1.0,NaN,NaN,NaN,NaN,NaN
3,1.0,NSE_FO,NIFTY,NSE,2025-09-09,CE,NIFTY,NIFTY,NSE_FO|40579,75.0,...,INDEX,INDEX,NIFTY 24750 CE 09 SEP 25,24750.0,1.0,NaN,NaN,NaN,NaN,NaN
4,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40578,75.0,...,INDEX,INDEX,NIFTY 24700 PE 09 SEP 25,24700.0,1.0,NaN,NaN,NaN,NaN,NaN
5,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40580,75.0,...,INDEX,INDEX,NIFTY 24750 PE 09 SEP 25,24750.0,1.0,NaN,NaN,NaN,NaN,NaN
6,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40582,75.0,...,INDEX,INDEX,NIFTY 24800 PE 09 SEP 25,24800.0,1.0,NaN,NaN,NaN,NaN,NaN
7,1.0,NSE_FO,NIFTY,NSE,2025-09-09,CE,NIFTY,NIFTY,NSE_FO|40581,75.0,...,INDEX,INDEX,NIFTY 24800 CE 09 SEP 25,24800.0,1.0,NaN,NaN,NaN,NaN,NaN
8,1.0,NSE_FO,NIFTY,NSE,2025-09-09,PE,NIFTY,NIFTY,NSE_FO|40571,75.0,...,INDEX,INDEX,NIFTY 24600 PE 09 SEP 25,24600.0,1.0,NaN,NaN,NaN,NaN,NaN
9,1.0,NSE_FO,NIFTY,NSE,2025-09-09,CE,NIFTY,NIFTY,NSE_FO|40570,75.0,...,INDEX,INDEX,NIFTY 24600 CE 09 SEP 25,24600.0,1.0,NaN,NaN,NaN,NaN,NaN


In [51]:
df_futureOptions.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   weekly             12 non-null     float64
 1   segment            12 non-null     object 
 2   name               12 non-null     object 
 3   exchange           12 non-null     object 
 4   expiry             12 non-null     object 
 5   instrument_type    12 non-null     object 
 6   asset_symbol       12 non-null     object 
 7   underlying_symbol  12 non-null     object 
 8   instrument_key     12 non-null     object 
 9   lot_size           12 non-null     float64
 10  freeze_quantity    12 non-null     float64
 11  exchange_token     12 non-null     int64  
 12  minimum_lot        12 non-null     float64
 13  asset_key          12 non-null     object 
 14  underlying_key     12 non-null     object 
 15  tick_size          12 non-null     float64
 16  asset_type         12 non-nu

CONNECT TO WEBSOCKET LIVE FEED

Fetch All Instrument Keys to subscribe

In [56]:
# ActivateMarketFeed
# token_list = [99926000] is a NIFTY 50 token to get ltp of NIFTY 50 index
token_list = list(set(df_futureOptions['instrument_key'].tolist()))
print(token_list)


['NSE_FO|40582', 'NSE_FO|40577', 'NSE_FO|40578', 'NSE_FO|40573', 'NSE_FO|40580', 'NSE_FO|40570', 'NSE_FO|40572', 'NSE_FO|40571', 'NSE_FO|40584', 'NSE_FO|40579', 'NSE_FO|40583', 'NSE_FO|40581']


In [55]:
access_token= CredentialUpstox.ACCESS_TOKEN
import time



feed_col = ['Token','Open', 'High', 'Low', 'Close', 'Ltp', 'Vol', 'Oi', 'theta', 'delta', 'gamma', 'vega', 'rho', 'iv', 'vtt']
df_feed = pd.DataFrame(columns=feed_col)


def on_message(message):
    print('inside message')
    print(message)
    
    feeds = message
    
    for token, feed_data in feeds.items():
        marketFF = feed_data.get("fullFeed", {}).get("marketFF", {})
        
        # Extract values safely
        ltpc = marketFF.get("ltpc", {})
        ohlc_list = marketFF.get("marketOHLC", {}).get("ohlc", [])
        greeks = marketFF.get("optionGreeks", {})
        
        # Use daily OHLC if available
        ohlc = next((item for item in ohlc_list if item.get("interval") == "1d"), {})
        
        row = {
            "Token": token,
            "Open": float(ohlc.get("open", 0)),
            "High": float(ohlc.get("high", 0)),
            "Low": float(ohlc.get("low", 0)),
            "Close": float(ohlc.get("close", 0)),
            "Ltp": float(ltpc.get("ltp", 0)),
            "Vol": float(ohlc.get("vol", 0)),
            "Oi": float(marketFF.get("oi", 0)),
            "theta": float(greeks.get("theta", 0)),
            "delta": float(greeks.get("delta", 0)),
            "gamma": float(greeks.get("gamma", 0)),
            "vega": float(greeks.get("vega", 0)),
            "rho": float(greeks.get("rho", 0)),
            "iv": float(marketFF.get("iv", 0)),
            "vtt": float(marketFF.get("vtt", 0)),
        }
        
        # Update if token exists, else append
        if token in df_feed["Token"].values:
            df_feed.loc[df_feed["Token"] == token, list(row.keys())] = list(row.values())
        else:
            df_feed = pd.concat([df_feed, pd.DataFrame([row])], ignore_index=True)
            
    print(df_feed)
    

def on_error(error):
    print("Error:", error)

def on_close():
    print("WebSocket closed")

def main():
    # Use your existing access_token
    configuration = upstox_client.Configuration()
    configuration.access_token = access_token

    # Replace with your desired instrument key and mode
    instrument_key = token_list  # e.g., 'NSE_INDEX|Nifty 50'
    mode = 'full_d30'  # full_d30, ltpc, full, etc.

    streamer = upstox_client.MarketDataStreamerV3(
        upstox_client.ApiClient(configuration),
        instrument_key,
        mode=mode
    )

    streamer.on("message", on_message)
    streamer.on("error", on_error)
    streamer.on("close", on_close)

    print("Connecting to Upstox Market Data Stream...")
    streamer.connect()
    while True:
        time.sleep(1)
    

